# Прогнозирование оттока клиентов банка

Необходимо построить модель классификации для прогнозирования оттока клиентов банка, опираясь на исторические данные о поведении клиентов и расторжении договоров с банком. Показатель F1-меры нужно довести как минимум до 0.59.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

* [1. Подготовка данных](#first_bullet)
    * [1.1. Импорт библиотек](#second_bullet)
    * [1.2. Чтение файла и изучение данных](#third_bullet)
    * [1.3. Предобработка данных](#fourth_bullet)
    * [1.4. Преобразование признаков](#fifth_bullet)
    * [1.5. Разбиение данных на выборки](#sixth_bullet)
    * [1.6. Масштабирование признаков](#seventh_bullet)
* [2. Исследование задачи](#eighth_bullet)
* [3. Борьба с дисбалансом](#ninth_bullet)
    * [3.1. Взвешивание классов](#tenth_bullet)
    * [3.2. Увеличение выборки *(upsampling)*](#eleventh_bullet)
    * [3.3. Уменьшение выборки *(downsampling)*](#twelfth_bullet)
    * [3.4. Настройка гиперпараметров](#thirteenth_bullet)
* [4. Тестирование модели](#fourteenth_bullet)
* [5. Вывод](#fifteenth_bullet)

# 1. Подготовка данных<a id="first_bullet"></a>

### 1.1. Импорт библиотек<a id="second_bullet"></a>

In [22]:
import matplotlib.pyplot as plt
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

import re
import statistics

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', -1)

### 1.2. Чтение файла и изучение данных<a id="third_bullet"></a>

Прочитаем файл и сохраним данные в переменной *data*. Выведем на экран первые и последние 5 строк датафрейма.

In [40]:
data = pd.read_csv('Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [41]:
data.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,NaN,130142.79,1,1,0,38190.78,0


Можно отметить, что:<a id="bullet"></a>
- датафрейм содержит как качественные (например *Surname*, *Geography* и др.), так и количественные признаки (*Age*, *Balance* и др.)
- есть признаки, которые  в рамках нашей задачи не играют важной роли, их можно будет удалить и таким образом облегчить работу будущей модели. К таким признакам можно отнести следующие признаки: *RowNumber*, *CustomerId* и *Surname*
- датафрейм содержит пропуски (см. строку с индексом 9999). Можно предположить, что пропуски появились по одной из этих причин:
    - клиент оставил графу пустой, так как не имеет недвижимость (в таком случае пропуски можно заполнить 0)
    - клиент просто не захотел ответить на этот вопрос и оставил графу пустой (тогда пропуски можно заполнить средним арифметическим или медианой).

Методом *info()* выведем общую информацию о датафрейме и посмотрим, к какому типу данных принадлежит каждый из признаков.

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Датафрейм *df* содержит следующие столбцы:

**Признаки**

- индекс строки в данных
- уникальный идентификатор клиента
- фамилия
- кредитный рейтинг
- страна проживания
- пол
- возраст
- количество недвижимости у клиента
- баланс на счёте
- количество продуктов банка, используемых клиентом
- наличие кредитной карты
- активность клиента
- предполагаемая зарплата

**Целевой признак**
- факт ухода клиента («ушел» — 1, «не ушел» — 0).

Датафрейм содержит 13 признаков (3 из них — качественные, а 10 — количественные) и 10000 объектов, признак *Tenure* содержит пропуски.

Методом *describe()* выведем более подробную информацию о количественных признаках.

In [43]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Можно отметить, что:
- признаки *CreditScore*, *Age*, *Tenure*, *Balance* и *EstimatedSalary* имеют достаточно большой разброс, о чем свидетельствует как сравнительно высокие показатели стандартного отклонения, так и большая разница между минимальными и максимальными значениями.
- масштаб признаков разный, поэтому, перед тем как приступить к обучению модели, необходимо их масштабировать.

### 1.3. Предобработка данных<a id="fourth_bullet"></a>

Для удобства названия столбцов приведем в такой формат:
- отделим слова нижним подчёркиванием
- регистр текста заменим на нижний.

In [44]:
data.columns = data.columns.str.replace(r"([A-Z])", r" \1").str.lower().str.replace(' ', '_').str[1:]
data.head()

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Помним, что признак *tenure* (количество недвижимости у клиента) содержит пропуски, их необходимо заполнить.

In [45]:
print('К-во пропусков в признаке tenure: ', data['tenure'].isnull().sum())

К-во пропусков в признаке tenure:  909


Посмотрим на распределение признака *tenure*.

In [46]:
tenure = data['tenure'].value_counts().to_frame()
tenure.reset_index(inplace=True)
tenure.columns = ['tenure_cnt', 'count']
tenure

,tenure_cnt,count
0,1.0,952
1,2.0,950
2,8.0,933
3,3.0,928
4,5.0,927
5,7.0,925
6,4.0,885
7,9.0,882
8,6.0,881
9,10.0,446


In [47]:
fig = go.Figure([go.Bar(x = tenure['tenure_cnt'], y=tenure['count'])])
fig.update_layout(height = 450, showlegend = False, title_text = "Распределение признака tenure")
fig.show()

Видим, что меньше всего клиентов, которые:
- не имеют недвижимость
- имеют 10 объектов недвижимости.

Похожее количество клиентов имеют от 2 до 9 объектов недвижимости.

В обработке пропусков можно действовать 2-мя способами:
- заполнить пропуски на 0 и предположить, что те клиенты, которые не имеют недвижимость, оставили эту графу пустой
- заполнить пропуски медианой или средним арифметическим.

Выведем на экран среднее арифметическое и медиану.

In [48]:
print('Среднее арифметическое:', data['tenure'].mean())
print('Медиана:', data['tenure'].median())

Среднее арифметическое: 4.997690023099769
Медиана: 5.0


Медиана и среднее арифметическое почти одинаковы и достаточно высоки.

Более вероятно, что у клиентов, которые оставили графу с вопросом о количестве объектов недвижимости пустым, нету недвижимости, поэтому заполним пропуски нулями.

In [49]:
data['tenure'] = data['tenure'].fillna(0)
data['tenure'].value_counts()

0.0     1291
1.0     952 
2.0     950 
8.0     933 
3.0     928 
5.0     927 
7.0     925 
4.0     885 
9.0     882 
6.0     881 
10.0    446 
Name: tenure, dtype: int64

Заменим тип данных признака *tenure* на *int*.

In [50]:
data['tenure'] = data['tenure'].astype('int')
data[['tenure']].dtypes

tenure    int64
dtype: object

Удалим признаки *row_number*, *customer_id*, *surname*.

In [51]:
df = data.drop(['row_number', 'customer_id', 'surname'], axis=1)
df.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Осталось 11 признаков: 2 качественных (*geography* и *gender*) и 9 количественных (*credit_score*, *age*, *tenure*, *balance*, *num_of_products*, *has_cr_card*, *is_active_member*, *estimated_salary*, *exited*).

Перед тем как перейти к преобразованию признаков, проверим не содержит ли датафрейм дубликаты.

In [52]:
print('Количество дубликатов:', df.duplicated().sum())

Количество дубликатов: 0


### 1.4. Преобразование признаков<a id="fifth_bullet"></a>

Для того, чтобы преобразовать категориальные признаки в численные, вызовем функцию *get_dummies()* с аргументом *drop_first*. Такой подход позволит не попасть в *дамми-ловушку*, так как не создает большое количество фиктивных признаков.

In [53]:
df_ohe = pd.get_dummies(df, drop_first=True)
df_ohe = df_ohe.rename(columns = {'geography_Germany': 'geography_germany',
                                  'geography_Spain': 'geography_spain',
                                  'gender_Male': 'gender_male'}, inplace = False)
df_ohe.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_germany,geography_spain,gender_male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


Проверка результата.

In [54]:
df_ohe.dtypes

credit_score         int64  
age                  int64  
tenure               int64  
balance              float64
num_of_products      int64  
has_cr_card          int64  
is_active_member     int64  
estimated_salary     float64
exited               int64  
geography_germany    uint8  
geography_spain      uint8  
gender_male          uint8  
dtype: object

### 1.5. Разбиение данных на выборки<a id="sixth_bullet"></a>

Так как спрятанной тестовой выборки у нас нету, разделим исходные данные на три выборки: обучающую, валидационную и тестовую. Так как размеры тестовой и валидационной выборок обычно равны, разделим данные в соотношении 3:1:1.

Сначала методом train_test_split разделим исходные данные на обучающую (60%) и валидационную выборку (40%). После этого разделим валидационную выборку пополам — на валидационную и тестовую выборки. Таким образом каждая из этих 2-х выборок составит 20% из всех данных *df_ohe*.

In [55]:
target = df_ohe['exited']
features = df_ohe.drop('exited', axis=1)

In [56]:
features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                              target,
                                                                              train_size=0.60,
                                                                              test_size=0.40,
                                                                              random_state=123,
                                                                              stratify=target)

In [57]:
features_valid, features_test, target_valid, target_test = train_test_split(features_valid,
                                                                            target_valid,
                                                                            train_size=0.50,
                                                                            test_size=0.50,
                                                                            random_state=123,
                                                                            stratify=target_valid)

Проверка результата.

In [58]:
samples = {'Размер обучающей выборки' : features_train,
          'Размер валидационной выборки': features_valid,
          'Размер тестовой выборки': features_test}

for key, value in samples.items():
    print(key + ':', value.shape)

Размер обучающей выборки: (6000, 11)
Размер валидационной выборки: (2000, 11)
Размер тестовой выборки: (2000, 11)


Как уже отметили ранее, масштаб признаков очень сильно отличается. Это может отрицательно повлиять на работу будущей модели (алгоритм может решить, что признак с более высокими значениями более важен чем другие признаки), поэтому признаки необходимо масштабировать.

### 1.6. Масштабирование признаков<a id="seventh_bullet"></a>

In [59]:
df_ohe.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_germany,geography_spain,gender_male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


Приведем признаки к одному масштабу. Для этого будем использовать структуру для стандартизации данных из библиотеке *sklearn* — *StandardScaler*.

Создадим объект структуры *StandardScaler* и настроим его на обучающих данных.

In [60]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

StandardScaler()

Преобразуем обучающую, валидационную и тестовую выборки функцией *transform()*.

In [61]:
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Проверка результата.

In [62]:
features_train.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_germany,geography_spain,gender_male
6255,-1.081763,1.561078,-1.463231,0.567612,2.552230,1,0,-1.444757,1,0,1
7141,1.083241,-0.091611,-1.140673,0.097622,-0.926107,0,1,-1.559126,1,0,0
3824,0.167278,0.491691,-1.463231,0.751314,-0.926107,0,0,-1.476028,0,0,1
1901,1.509997,2.727681,0.149559,-1.219982,0.813061,0,1,-0.786010,0,1,1
2886,-2.101813,-0.188828,0.472117,-1.219982,0.813061,1,0,-0.312019,0,0,0


Сейчас признаки имеют одинаковый масштаб. Можем перейти к исследованию задачи.

### Результат:
    
- изучили данные и сделали предобработку данных (заменили тип данных и заполнили пропуски для признака tenure, проверили данные на дубликаты и переименовали признаки)
- преобразовали категориальные признаки в численные
- разделили исходные данные на три выборки: обучающую, валидационную и тестовую в соотношении 3:1:1.
- масштабировали признаки.

# 2. Исследование задачи<a id="eighth_bullet"></a>

В рамках проекта нам предстоит решить задачу бинарной классификации: спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Но перед тем, как приступить к задаче, необходимо:

- исследовать баланс классов
- обучить модели без учёта дисбаланса.

Проверим баланс классов.

In [63]:
values = df_ohe['exited'].value_counts()
values = values.to_frame().reset_index()
values.columns = ['Boolean', 'Count']
values['Percentage'] = df_ohe['exited'].value_counts(normalize=True)
pd.DataFrame(values)
values

,Boolean,Count,Percentage
0,0,7963,0.7963
1,1,2037,0.2037


Визуализируем результат.

In [64]:
def visualize_class_balance(x, y, values, labels):

    '''

    Функция для визуализации баланса классов (в абсолютных величинах и в процентах).

    '''

    fig = make_subplots(rows = 1, cols = 2,
                    specs = [[{"type": "bar"}, {"type": "pie"}]],
                    subplot_titles = ('Доля клиентов, в зависимости от класса',
                                    'Доля клиентов, в зависимости от класса'))

    fig.add_trace(go.Bar(x = x, y = y), row = 1, col = 1)
    fig.add_trace(go.Pie(values = values, labels = labels), row = 1, col = 2)

    fig.update_layout(height = 400, showlegend = False, title_text = "Факт ухода клиента, где 1-ушел, 0-остался")
    fig.show()

visualize_class_balance(values['Boolean'], values['Count'], values['Percentage'], values['Boolean'])

Видим, что классы не сбалансированы.

Обучим разные модели без учёта дисбаланса. Так как нам предстоит решить задачу бинарной классификации, рассмотрим следующие модели классификации:
- дерево принятия решений *(Decision Tree Classifier)*
- случайный лес *(Random Forest Classifier)*
- логистическая регрессия *(Logistic Regression)*.

In [65]:
models = [LogisticRegression(random_state=123),
          DecisionTreeClassifier(random_state=123),
          RandomForestClassifier(random_state=123)]

results_imbalanced = []

for model in models:
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)

    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])

    results_imbalanced.append({'Model': model.__class__.__name__,
                               'accuracy_imb': accuracy,
                               'f1_imb': f1,
                               'ROC AUC_imb': roc_auc})

In [66]:
pd.DataFrame(results_imbalanced).style.highlight_max(color = 'lightgreen', axis = 0)

,Model,accuracy_imb,f1_imb,ROC AUC_imb
0,LogisticRegression,0.804000,0.302491,0.731555
1,DecisionTreeClassifier,0.789500,0.493381,0.682758
2,RandomForestClassifier,0.856000,0.556923,0.846490


Наилучший результат без учета дисбаланса показала модель *случайного леса*.

### Результат:
    
- проверили и сделали визуализацию баланса классов
    
- обучили следующие модели без учёта дисбаланса:
    - дерево принятия решений
    - случайный лес
    - логистическая регрессия.

- для каждой из моделей вывели следующие метрики: *accuracy_score*, *f1_score* и *roc_auc_score*. По всем метрикам наилучший результат по показала модель случайного леса (а1_score = 0.54, roc_auc_score = 0.83).



# 3. Борьба с дисбалансом<a id="ninth_bullet"></a>

Существует несколько способов борьбы с дисбалансом, в рамках этого проекта будем применять следующие:
- взвешивание классов (объектам редкого класса придается больший вес)
- увеличение выборки (*upsampling*)
- уменьшение выборки (*downsampling*).

### 3.1. Взвешивание классов<a id="tenth_bullet"></a>

Обучим модели, указывая гиперпараметр *class_weight* = *balanced*.

In [67]:
models_balanced = [LogisticRegression(random_state=123, class_weight='balanced'),
                   DecisionTreeClassifier(random_state=123, class_weight='balanced'),
                   RandomForestClassifier(random_state=123, class_weight='balanced')]

results_balanced = []

for model in models_balanced:
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)

    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])

    results_balanced.append({'Model': model.__class__.__name__,
                             'accuracy_balanced': accuracy,
                             'f1_balanced': f1,
                             'ROC AUC score_balanced': roc_auc})

Сохраним результат в датафрейме *final_results*. Методом *merge()* объединим полученные результаты с результатами, которые получили, когда обучали модели без учета дисбаланса классов.

In [68]:
final_results = pd.DataFrame(results_imbalanced).merge(pd.DataFrame(results_balanced), on='Model')
final_results = final_results.reindex(sorted(final_results.columns), axis=1)
final_results.style.highlight_max(color = 'lightgreen', axis = 0)

,Model,ROC AUC score_balanced,ROC AUC_imb,accuracy_balanced,accuracy_imb,f1_balanced,f1_imb
0,LogisticRegression,0.734507,0.731555,0.691500,0.804000,0.460192,0.302491
1,DecisionTreeClassifier,0.679802,0.682758,0.793500,0.789500,0.490752,0.493381
2,RandomForestClassifier,0.845489,0.846490,0.859000,0.856000,0.563467,0.556923


Наилучший результат опять показала модель случайного леса, но интересно, что показатель *f1_score* стал ниже для всех моделей, кроме модели логистической регрессии.

### 3.2. Увеличение выборки *(upsampling)*<a id="eleventh_bullet"></a>

Идея данного подхода заключается в том, чтобы преобразовать объекты редкого класса так, чтобы они стали не такими редкими.

Преобразование проходит в несколько этапов:
- разделяем обучающую выборку на отрицательные и положительные объекты
- копируем несколько раз положительные объекты
- создаем новую обучающую выборку с учётом полученных данных
- перемешиваем данные.

In [69]:
def upsample(features, target, repeat):

    '''

    Функция для увеличение выборки (upsampling).
    Как аргументы ф-я принимает признаки (features), целевой признак (target) и к-во повторений (repeat).

    Ф-я работает по следующему принципу:
        - разделяет обучающую выборку на отрицательные и положительные объекты
        - копирует несколько раз положительные объекты
        - создает новую обучающую выборку с учётом полученных данных
        - перемешивает данные
        - возвращает преобразованные выборки.

    '''

    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=123)

    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

Выведем на экран размеры новых выборок и при помощи ранее созданной ф-ии *visualize_class_balance()* посмотрим, как изменился баланс классов.

In [70]:
print(features_upsampled.shape)
print(target_upsampled.shape)

(9666, 11)
(9666,)


In [71]:
values = target_upsampled.value_counts()
values = values.to_frame().reset_index()
values.columns = ['Boolean', 'Count']
values['Percentage'] = values['Count'] / sum(values['Count'])
pd.DataFrame(values)
values

visualize_class_balance(values['Boolean'], values['Count'], values['Percentage'], values['Boolean'])

Сейчас выборки сбалансированы, к-во целевого признака существенно увеличилось.

Обучим модели на сбалансированных данных.

In [72]:
results_upsampling = []

for model in models:
    model.fit(features_upsampled, target_upsampled)
    predictions_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)

    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])

    results_upsampling.append({'Model': model.__class__.__name__,
                               'accuracy_up': accuracy,
                               'f1_up': f1,
                               'ROC AUC score_up': roc_auc})

Сохраним результаты в датафрейме *final_results*.

In [73]:
final_results = final_results.merge(pd.DataFrame(results_upsampling), on='Model')
final_results = final_results.reindex(sorted(final_results.columns), axis=1)
final_results.style.highlight_max(color = 'lightgreen', axis = 0)

,Model,ROC AUC score_balanced,ROC AUC score_up,ROC AUC_imb,accuracy_balanced,accuracy_imb,accuracy_up,f1_balanced,f1_imb,f1_up
0,LogisticRegression,0.734507,0.734518,0.731555,0.691500,0.804000,0.686500,0.460192,0.302491,0.458081
1,DecisionTreeClassifier,0.679802,0.682001,0.682758,0.793500,0.789500,0.797000,0.490752,0.493381,0.495025
2,RandomForestClassifier,0.845489,0.842090,0.846490,0.859000,0.856000,0.851500,0.563467,0.556923,0.597015


Видим, что модель случайного леса стала прогнозировать лучше. Сейчас *f1-score* = 0.57, а *roc_auc_score* = 0.815 (что ниже, чем на несбалансированной выборке).

Модель логистической регрессии лучше отработала на данных, сбалансированных при помощи гиперпараметра *class_weight = balanced*. При этом показатель *roc_auc_score* почти не изменился.

Дерево принятия решений показывает почти одинаковый результат *f1_score* на данных, сбалансированных при помощи увеличение выборки (0.49) и на несбалансированных данных (0.49).

### 3.3. Уменьшение выборки *downsampling*<a id="twelfth_bullet"></a>

Суть этого подхода в том, чтобы сделать объекты частого класса не такими частыми:
- разделяем обучающую выборку на отрицательные и положительные объекты
- случайным образом отбрасываем часть из отрицательных объектов
- создаем новую обучающую выборку с учётом полученных данных
- перемешиваем данные.

In [74]:
def downsample(features, target, fraction):

    '''

    Функция для увеличение выборки (downsampling).
    Как аргументы ф-я принимает признаки (features), целевой признак (target) и доля отрицательных объектов,
    которые нужно сохранить (fraction).

    Ф-я работает по следующему принципу:
        - разделяет обучающую выборку на отрицательные и положительные объекты
        - случайным образом отбрасывает часть из отрицательных объектов
        - создает новую обучающую выборку с учётом полученных данных
        - перемешивает данные
        - возвращает преобразованные выборки.

    '''

    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=123)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=123)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=123)
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

Выведем на экран размеры переменных *features_downsampled* и *target_downsampled* и посмотрим на баланс классов.

In [75]:
print(features_downsampled.shape)
print(target_downsampled.shape)

(2416, 11)
(2416,)


In [76]:
values = target_downsampled.value_counts()
values = values.to_frame().reset_index()
values.columns = ['Boolean', 'Count']
values['Percentage'] = values['Count'] / sum(values['Count'])
pd.DataFrame(values)
values

visualize_class_balance(values['Boolean'], values['Count'], values['Percentage'], values['Boolean'])

Выборки сбалансированы, можем обучить модели на полученных данных.

In [77]:
results_downsampling = []

for model in models:
    model.fit(features_downsampled, target_downsampled)
    predictions_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)

    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])

    results_downsampling.append({'Model': model.__class__.__name__,
                                 'accuracy_down': accuracy,
                                 'f1_down': f1,
                                 'ROC AUC score_down': roc_auc})

Добавим результат в датафрейме *final_results* и посмотрим, какая из моделей показывает наилучший результат по таким метрикам как *f1_score* и *roc_auc_score*.

In [78]:
final_results = final_results.merge(pd.DataFrame(results_downsampling), on='Model')
final_results = final_results.reindex(sorted(final_results.columns), axis=1)
final_results.style.highlight_max(color = 'lightgreen', axis = 0)

,Model,ROC AUC score_balanced,ROC AUC score_down,ROC AUC score_up,ROC AUC_imb,accuracy_balanced,accuracy_down,accuracy_imb,accuracy_up,f1_balanced,f1_down,f1_imb,f1_up
0,LogisticRegression,0.734507,0.733288,0.734518,0.731555,0.691500,0.682000,0.804000,0.686500,0.460192,0.456410,0.302491,0.458081
1,DecisionTreeClassifier,0.679802,0.692069,0.682001,0.682758,0.793500,0.695500,0.789500,0.797000,0.490752,0.479042,0.493381,0.495025
2,RandomForestClassifier,0.845489,0.840597,0.842090,0.846490,0.859000,0.758000,0.856000,0.851500,0.563467,0.555147,0.556923,0.597015


Наилучший результат показывает модель случайного леса, но интересно посмотреть, на какой выборке каждая из моделей показала лучший результат метрики *f1_score*.  

In [79]:
final_results.style.highlight_max(subset = final_results.columns[-4:], color = 'lightgreen', axis = 1)

,Model,ROC AUC score_balanced,ROC AUC score_down,ROC AUC score_up,ROC AUC_imb,accuracy_balanced,accuracy_down,accuracy_imb,accuracy_up,f1_balanced,f1_down,f1_imb,f1_up
0,LogisticRegression,0.734507,0.733288,0.734518,0.731555,0.691500,0.682000,0.804000,0.686500,0.460192,0.456410,0.302491,0.458081
1,DecisionTreeClassifier,0.679802,0.692069,0.682001,0.682758,0.793500,0.695500,0.789500,0.797000,0.490752,0.479042,0.493381,0.495025
2,RandomForestClassifier,0.845489,0.840597,0.842090,0.846490,0.859000,0.758000,0.856000,0.851500,0.563467,0.555147,0.556923,0.597015


Видим, что модель логистической регрессии лучше работает на данных, сбалансированных при помощи гиперпараметра *class_weight = balanced* (f1_score = 0.46). Остальные модели лучше работают на данных, преобразованных при помощи увеличение выборки.

На следующем этапе проекта обучим каждую из моделей на тех данных, на которых она показала наилучший результат метрики *f1_score*. Для каждой модели подберем разные гиперпараметры и попробуем улучшить ее работу.

### 3.4. Настройка гиперпараметров<a id="thirteenth_bullet"></a>

#### Логистическая регрессия

Настроим гиперпараметры и обучим модель логистической регрессии на данных, сбалансированных при помощи гиперпараметра *class_weight = balanced*.

In [80]:
results_lr = []

penalty_l1 = LogisticRegression(random_state=123, class_weight='balanced', solver='liblinear', penalty='l1')
penalty_l2 = LogisticRegression(random_state=123, class_weight='balanced', solver='lbfgs', penalty='l2')

models_lr = [penalty_l1, penalty_l2]

for model in models_lr:
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)

    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])

    results_lr.append({'Model': model.__class__.__name__ + '_' + model.penalty ,
                       'Hyperparameters': {'random_state': 123,
                                          'class_weight': model.class_weight,
                                          'solver': model.solver,
                                          'penalty': model.penalty},
                       'Accuracy': accuracy,
                       'F1 score': f1,
                       'ROC AUC score': roc_auc})

pd.DataFrame(results_lr)

,Model,Hyperparameters,Accuracy,F1 score,ROC AUC score
0,LogisticRegression_l1,"{'random_state': 123, 'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l1'}",0.758,0.555147,0.734554
1,LogisticRegression_l2,"{'random_state': 123, 'class_weight': 'balanced', 'solver': 'lbfgs', 'penalty': 'l2'}",0.758,0.555147,0.734507


Результат почти идентичен, но модель логистической регрессии с гиперпараметром *penalty = l1* дает незначительно лучше результат по метрике *roc_auc_score*. Результат метрики *f1_score* в обоих случаях = 0.539.

Сохраним результат модель логистической регрессии с гиперпараметром *penalty = l1* в переменной *best_results*.  

In [81]:
best_results = []
best_results.append(pd.DataFrame(results_lr).loc[0])

#### Дерево принятия решений

Помним, что данная модель лучше работала на данных, преобразованных при помощи увеличение выборки. В цикле настроим
гиперпараметр *max_depth* (глубина дерева) и попробуем улучшить работу модели.

In [85]:
results_dtc = []

for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)

    model.fit(features_upsampled, target_upsampled)
    predictions_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)

    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])

    results_dtc.append({'Model': 'DecisionTreeClassifier',
                        'Hyperparameters': {'random_state': 123, 'max_depth':depth},
                        'Accuracy': accuracy,
                        'F1 score': f1,
                        'ROC AUC score': roc_auc})

pd.DataFrame(results_dtc)

,Model,Hyperparameters,Accuracy,F1 score,ROC AUC score
0,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 1}",0.7100,0.462963,0.673835
1,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 2}",0.7200,0.496403,0.735933
2,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 3}",0.7200,0.496403,0.785846
3,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 4}",0.7485,0.544796,0.811886
4,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 5}",0.7365,0.550725,0.833776
5,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 6}",0.7605,0.559338,0.826147
6,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 7}",0.7625,0.548908,0.813561
7,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 8}",0.7615,0.540905,0.780740
8,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 9}",0.7580,0.528265,0.762914
9,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 10}",0.7520,0.510848,0.735242


Наилучший результат метрики *f1_score* модель показывает при глубине дерева 6. Добавим эту модель в переменную *best_results*.

In [86]:
best_results.append(pd.DataFrame(results_dtc).loc[5])

Осталось настроить гиперпараметри для модели случайного леса.

#### Случайный лес

В цикле настроим гиперпараметры *max-depth* (глубина деревьев) и *n_estimators* (количество деревьев в *лесу*). Результат сохраним в переменной *results_rfc_up*.

In [87]:
%%time
results_rfc_up = []

for depth in range(1,15):

    for estimator in range(10, 101, 10):

        model = RandomForestClassifier(random_state=123,
                                       n_estimators=estimator,
                                       max_depth=depth)

        model.fit(features_upsampled, target_upsampled)
        predictions_valid = model.predict(features_valid)
        probabilities_valid = model.predict_proba(features_valid)

        accuracy = accuracy_score(target_valid, predictions_valid)
        f1 = f1_score(target_valid, predictions_valid)
        roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])

        results_rfc_up.append({'Model': 'RandomForestClassifier',
                               'Hyperparameters': {'random_state': 123,
                                                   'n_estimators': estimator,
                                                   'max_depth':depth},
                               'Accuracy': accuracy,
                               'F1 score': f1,
                               'ROC AUC score': roc_auc})

CPU times: user 1min 12s, sys: 161 ms, total: 1min 12s
Wall time: 1min 16s


Лучший результат дает следующая комбинация гиперпараметров:

In [88]:
df_rfc_up = pd.DataFrame.from_dict(results_rfc_up)
df_rfc_up[df_rfc_up['F1 score']==df_rfc_up['F1 score'].max()]

,Model,Hyperparameters,Accuracy,F1 score,ROC AUC score
91,RandomForestClassifier,"{'random_state': 123, 'n_estimators': 20, 'max_depth': 10}",0.818,0.6,0.848249


Видим, что наилучший результат показала модель случайного леса со следующей комбинацией гиперпараметров: *n_estimators* = 20 и *max_depth* = 10. Добавим этот результат в переменную *best_results* и выведем ее на экран.

In [90]:
best_results.append(pd.DataFrame(df_rfc_up).loc[91])
pd.DataFrame(best_results)

,Model,Hyperparameters,Accuracy,F1 score,ROC AUC score
0,LogisticRegression_l1,"{'random_state': 123, 'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l1'}",0.7580,0.555147,0.734554
5,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 6}",0.7605,0.559338,0.826147
91,RandomForestClassifier,"{'random_state': 123, 'n_estimators': 20, 'max_depth': 10}",0.8180,0.600000,0.848249
91,RandomForestClassifier,"{'random_state': 123, 'n_estimators': 20, 'max_depth': 10}",0.8180,0.600000,0.848249


Видим, что и после настройки гиперпараметров наилучший результат показала модель случайного леса. Протестируем работу этой модели на тестовых данных.


### Результат:
    
- применили несколько способов борьбы с дисбалансом (взвешивание классов, увеличение выборки, уменьшение выборки ) и проверили, как преобразование данных влияет на способность модели предсказывать уход клиентов.

- выяснили, что наилучший показатель таких метрик как *f1_score* и *roc_auc_score* модель случайного леса и модель дерева принятия решений достигается на данных преобразованных при помощи увеличение выборки. До настроики гиперпараметров показатель метрики *f1_score* следующий:   
    - случайный лес: 0.576
    - дерево принятия решений: 0.495
    
- Модель логистической регрессии лучше работает на данных, сбалансированных при помощи гиперпараметра *class_weight = balanced* (f1_score = 0.46).
    
- Настройка гиперпараметров позволила улучшить показатель метрики *f1_score* для всех моделей:
    - случайный лес: 0.6
    - дерево принятия решений: 0.559
    - логистическая регрессия: 0.539.

Протестируем модель случайного леса с гиперпараметрами *n_estimators = 20* и *max_depth = 10* на тестовой выборке.


# 4. Тестирование модели<a id="fourteenth_bullet"></a>

Протестируем работу модели случайного леса со следующей комбинацией гиперпараметров: *n_estimators* = 20 и *max_depth* = 10.

In [91]:
model_final = RandomForestClassifier(random_state=123, n_estimators=20, max_depth=10)

model_final.fit(features_upsampled, target_upsampled)
predictions_final = model_final.predict(features_test)
probabilities_final = model_final.predict_proba(features_test)

accuracy_final = accuracy_score(target_test, predictions_final)
f1_final = f1_score(target_test, predictions_final)
roc_auc_final = roc_auc_score(target_test, probabilities_final[:,1])

print('accuracy', accuracy_final)
print('f1 score', f1_final)
print('roc_auc', roc_auc_final)

accuracy 0.8225
f1 score 0.6203208556149733
roc_auc 0.858824155434325


Проверим работу модели на расширенной выборке. Для этого:

- объединим тренировочную и валидационную выборки:

In [92]:
features_combined = pd.concat([features_train, features_valid])
target_combined = pd.concat([target_train, target_valid])

- увеличим выборки, при помощи функции *upsample()*:

In [93]:
features_upsampled_combined, target_upsampled_combined = upsample(features_combined, target_combined, 4)

In [94]:
print(features_upsampled_combined.shape)
print(target_upsampled_combined.shape)

(12890, 11)
(12890,)


- используем полученные выборки при обучении модели случайного леса с гиперпараметрами *n_estimators = 20* и *max_depth = 10*. Протестируем работу модели на тестовой выборке.

In [95]:
model_combined = RandomForestClassifier(random_state=123, n_estimators=20, max_depth=10)

model_combined.fit(features_upsampled_combined, target_upsampled_combined)

predictions_combined = model_combined.predict(features_test)
probabilities_combined = model_combined.predict_proba(features_test)

accuracy_combined = accuracy_score(target_test, predictions_final)
f1_combined = f1_score(target_test, predictions_combined)
roc_auc_combined = roc_auc_score(target_test, probabilities_combined[:,1])

print('accuracy', accuracy_combined)
print('f1 score', f1_combined)
print('roc_auc', roc_auc_combined)

accuracy 0.8225
f1 score 0.6153846153846154
roc_auc 0.8630063036842699


Построим график *ROC-кривой*.

In [96]:
fpr, tpr, thresholds = roc_curve(target_test, probabilities_final[:,1])

# ROC-кривая случайного леса
trace_rf = go.Scatter(x = fpr,y = tpr,
                      name = "Случайный лес: " + str(roc_auc_final),
                      line = dict(width = 2))

# ROC-кривая случайной модели
trace_random = go.Scatter(x = [0.0, 1.0], y = [0.0, 1.0],
                          name = 'Случайная модель',
                          line = dict(width = 2, dash = 'dot'))

data = [trace_rf, trace_random]
layout = go.Layout(dict(title = 'ROC-кривая',
                        height = 550, width = 900,
                        xaxis = dict(title = "Ложноположительные ответы (False Positive Rate)"),
                        yaxis = dict(title = "Истинно положительные ответы (True Positive Rate)")))

fig = go.Figure(data, layout=layout)
fig.show()

Проверим финальную модель на адекватность<a id="bullet2"></a>.

In [97]:
strategies = ['stratified', 'most_frequent', 'uniform']

dummy_results = []
for strategy in strategies:
    dc = DummyClassifier(strategy = strategy, random_state = 42)

    dc.fit(features_train, target_train)
    result = dc.score(features_test, target_test)
    dummy_results.append({strategy: result})

pd.DataFrame(dummy_results).style.highlight_max(color = 'lightgreen', axis = 1)

,stratified,most_frequent,uniform
0,0.664000,nan,nan
1,nan,0.796500,nan
2,nan,nan,0.492500


Видим, что модель случайного леса с гиперпараметрами n_estimators = 20 и max_depth = 10 работает лучше, чем стратегии случайного прогнозирования классификатора *DummyClassifier()*.

# 5. Вывод<a id="fifteenth_bullet"></a>

Проверка модели на тестовой выборке и на расширенной выборке показывает, что модель работает достаточно хорошо — показатель метрики *f1_score* удалось достигнуть выше, чем было заявлено в требованиях (минимальное требование к этой метрике было 0.59, наша модель дает результат 0.62).
    
Метрика *roc_auc_score* = 0.858, что является хорошим результатом.
    
Модель работает хорошо, цель проекта достигнута.